In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn


In [3]:
x = torch.ones(2, 10, 5)

In [4]:
from hw_ss.model.SpEX_plus import SpEXPlus

In [67]:
kw = {
    "L1": 20,
    "L2": 80,
    "L3": 160,
    "speech_encoder_out_channels": 256,
    "num_tcn_blocks_in_stack": 8,
    "num_stacked_tcn": 2,
    "num_res_net_blocks": 2,
    "spk_emb": 256,
    "num_spk": 101,
    "extractor_emb_dim": 256,
    "extractor_intermed_dim":512,
    "tcn_kernel_size": 3
}

In [68]:
model = SpEXPlus(**kw)

In [69]:
model

SpEXPlus(
  (speech_and_speaker_encoder): SpeechEncoder(
    (encoder_short): Conv1d(1, 256, kernel_size=(20,), stride=(10,))
    (encoder_middle): Conv1d(1, 256, kernel_size=(80,), stride=(10,))
    (encoder_long): Conv1d(1, 256, kernel_size=(160,), stride=(10,))
  )
  (channel_norm): ChannelWiseLayerNorm(
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (conv1x1_extractor): Conv1d(768, 256, kernel_size=(1,), stride=(1,))
  (tcn_extractors): SpeakerExtractor(
    (extractor): ModuleDict(
      (StackedTCN #1): StackedTCN(
        (STCN): ModuleDict(
          (FTCN): FTCNBlock(
            (block): Sequential(
              (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
              (1): PReLU(num_parameters=1)
              (2): GlobalLayerNorm()
              (3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), groups=512)
              (4): PReLU(num_parameters=1)
              (5): GlobalLayerNorm()
              (6): Conv1d(512, 256, ke

In [79]:
batch = torch.randn(3, 1, 100)
batch.shape

torch.Size([3, 1, 100])

In [80]:
s1, s2, s3, pred = model(batch, batch)

AFTER SPEECH AND SPEAKER: tensor([[[0.0000, 0.4008, 0.0000,  ..., 0.0000, 0.0775, 0.0000],
         [0.1084, 1.3018, 0.5662,  ..., 0.0000, 0.0000, 0.4259],
         [0.0000, 0.0000, 0.0000,  ..., 0.5513, 1.4991, 0.7461],
         ...,
         [0.8323, 0.0000, 0.0000,  ..., 0.1682, 1.1922, 0.5257],
         [0.2505, 0.7167, 0.4086,  ..., 0.0000, 0.0251, 0.1525],
         [1.0955, 0.0000, 0.3847,  ..., 1.2100, 0.8983, 0.0000]],

        [[0.6957, 0.0000, 0.0000,  ..., 0.4294, 0.9368, 0.0000],
         [0.2091, 0.1107, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.4659, 0.0000, 0.5078,  ..., 1.4263, 0.0000, 0.5699],
         ...,
         [0.6354, 0.3962, 0.0000,  ..., 0.4655, 0.0000, 1.5718],
         [0.5411, 0.0000, 0.0000,  ..., 0.0212, 0.0000, 1.0932],
         [0.0000, 0.9326, 0.0000,  ..., 0.0000, 0.0000, 1.0216]],

        [[0.0000, 0.2618, 0.3787,  ..., 0.4978, 0.1400, 0.0000],
         [0.3629, 0.0000, 0.0000,  ..., 0.5358, 0.0000, 0.0000],
         [1.0708, 0.1833, 0.5505

In [82]:
s1.shape

torch.Size([3, 1, 100])

In [95]:
def SISDRLoss(pred, target):
    pred_hat = pred - pred.mean()
    target_hat = target - target.mean()
    pred_t = torch.inner(pred_hat, target_hat) * target_hat / (target_hat ** 2).sum()
    target_t = pred_hat - pred_t
    
    ret = 20 * torch.log10(torch.norm(pred_t) / (torch.norm(target_t) + 1e-6))
    return ret

In [96]:
torch.ones(size=(10, 1, 3)) - torch.ones(size=(10, 1, 3)).mean(dim=(1, 2)).reshape(-1, 1)

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
   

In [97]:
SISDRLoss(torch.ones(size=(10, 1, 3)), torch.ones(size=(10, 1, 3)))

tensor(nan)

In [105]:
subsampled = subsample(batch, batch.shape)

In [106]:
lin = linear_proj(subsampled)

In [107]:
from feed_forward import ConformerFeedForwardLayer

In [108]:
ffl = ConformerFeedForwardLayer(512, 2, 0.1)

In [109]:
after_ffl = ffl(lin) + lin

In [110]:
attn = ConformerAttentionBlock(512, 8, 0.1)

In [111]:
after_attn = attn(after_ffl) + after_ffl

In [112]:
from conv import ConformerConvBlock

In [116]:
conv_ = ConformerConvBlock(in_chanels=512, exp_factor=2, kernel_size=31, padding=15, stride=1, dropout=0.1)

In [117]:
after_attn.shape

torch.Size([3, 31, 512])

In [118]:
after_conv = conv_(after_attn)


torch.Size([3, 1024, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])
torch.Size([3, 512, 31])


In [93]:
ffl_2 = ConformerFeedForwardLayer(512, 2, 0.1)

In [95]:
shit = (ffl_2(after_conv) + after_conv)

In [96]:
shit

torch.Size([3, 31, 512])

In [176]:
class ConvSubsample(nn.Module):
    def __init__(self, in_chanels, out_chanels):
        super().__init__()
        self.subsampler = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=out_chanels, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_chanels, out_channels=out_chanels, kernel_size=3, stride=2),
            nn.ReLU()
        )
    
    def forward(self, x):
        out = self.subsampler(x.unsqueeze(1))
        batch_size, channels, subsample_len, subsample_dim = out.shape

        out = out.permute(0, 2, 1, 3)
        out = out.contiguous().view(batch_size, subsample_len, channels * subsample_dim)
        return out
    


In [181]:
testing_jopa = ConvSubsample(1, 144)
linear = nn.Linear(144 * ((140 - 1)//2 - 1)//2, 144)

In [178]:
batch = torch.randn(3, 128, 140)
batch.shape

torch.Size([3, 128, 140])

In [186]:
out = testing_jopa(batch)
linear(out).shape

3 144 31 34
4896


torch.Size([3, 31, 144])

In [184]:
144 * (((140 - 1) // 2 - 1) // 2)

4896